## Import Libraries

In [1]:
from pymongo import MongoClient
from bson.son import SON
import json

In [2]:
import pprint

In [3]:
MONGODB_HOST = 'localhost'
MONGODB_PORT = 27017
DBS_NAME = 'mydb'
COLLECTION_NAME = 'projects'
FIELDS = "FeatureCollection"

In [4]:
client = MongoClient()
db = client.mydb
collection = db.projects

### Code

In [226]:
def dashboard_projects():
    connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
    collection = connection[DBS_NAME][COLLECTION_NAME]
    projects = collection.find_one({"type": "FeatureCollection"}, {"_id":0})

    return projects

In [256]:
dashboard_projects()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.965463853978, -37.8077644149636]},
   'properties': {'FIELD1': 0,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Old Melbourne Gaol Crime & Justice Experience'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.953478052064, -37.8242574266145]},
   'properties': {'FIELD1': 1,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Polly Woodside'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.967252924593, -37.8266045432686]},
   'properties': {'FIELD1': 2,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Australian Centre for Contemporary Art'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.968837288153, -37.8219947335768]},
   'properties'

In [295]:
# Call all documents
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
projects2 = collection.find_one({"type": "FeatureCollection"}, {"_id":0})

In [319]:
db = MongoClient()
latitude = -37.44609999
longitude = 144.96923385414
max_distance = 1000 #meters    

In [50]:
location = [144.965234, -37.8076]

In [43]:
db.collection.drop_indexes()

In [48]:
for idx in db.collection.list_indexes():
    print (idx)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'mydb.collection')])
SON([('v', 2), ('key', SON([('features.geometry.coordinates', '2dsphere')])), ('name', 'features.geometry.coordinates_2dsphere'), ('ns', 'mydb.collection'), ('2dsphereIndexVersion', 3)])


In [45]:
db.collection.create_index([( "features.geometry.coordinates", "2dsphere" )])

'features.geometry.coordinates_2dsphere'

In [785]:
## Unwind search

pipeline = [{"$unwind": "$features"},
            {"$match":{'features.geometry.coordinates':[144.965463853978, -37.8077644149636]}},
            {"$group": {"_id": "$features"}}]

pprint.pprint(list(collection.aggregate(pipeline)))

[]


In [20]:
gallery = [{"$unwind": "$features"},
            {"$match":{'features.properties.Sub Theme' : 'Art Gallery/Museum'}},
            {"$group": {"_id": "$features"}}]

pprint.pprint(list(collection.aggregate(gallery)))

[{'_id': {'geometry': {'coordinates': [144.969070431323, -37.8176107327202],
                       'type': 'Point'},
          'properties': {'FIELD1': 17,
                         'Feature Name': 'Australian Centre For The Moving '
                                         'Image (ACMI)',
                         'Sub Theme': 'Art Gallery/Museum',
                         'Theme': 'Place Of Assembly'},
          'type': 'Feature'}},
 {'_id': {'geometry': {'coordinates': [144.971521914952, -37.80460262432],
                       'type': 'Point'},
          'properties': {'FIELD1': 16,
                         'Feature Name': 'Royal Exhibition Building',
                         'Sub Theme': 'Art Gallery/Museum',
                         'Theme': 'Place Of Assembly'},
          'type': 'Feature'}},
 {'_id': {'geometry': {'coordinates': [144.979471339826, -37.8144604899837],
                       'type': 'Point'},
          'properties': {'FIELD1': 15,
                         'Feature

## Working code

In [60]:
pipeline = [
        {'$geoNear': {
            'near' : { 'type' : "Point", 'coordinates' : location },
             'distanceField': "dist.calculated",
             'maxDistance': 100,
             'includeLocs': "dist.location",
              'key': 'features.geometry.coordinates',
             'uniqueDocs': False,
             'spherical': True}},
    { "$unwind" : "$features" },
    { "$redact" : {   
        "$cond" : {
            "if" : { "$eq" : [ { "$cmp" : [ "$features.geometry.coordinates", "$dist.location" ] }, 0 ] },
            "then" : "$$KEEP",
            "else" : "$$PRUNE"
        }
    }
}
]

pprint.pprint(list(collection.aggregate(pipeline)))

[{'_id': ObjectId('5ca63a65122b9a6192720138'),
  'dist': {'calculated': 27.27001406965783,
           'location': [144.965463853978, -37.8077644149636]},
  'features': {'geometry': {'coordinates': [144.965463853978,
                                            -37.8077644149636],
                            'type': 'Point'},
               'properties': {'FIELD1': 0,
                              'Feature Name': 'Old Melbourne Gaol Crime & '
                                              'Justice Experience',
                              'Sub Theme': 'Art Gallery/Museum',
                              'Theme': 'Place Of Assembly'},
               'type': 'Feature'},
  'type': 'FeatureCollection'},
 {'_id': ObjectId('5ccd860ce39a5cbeb4b1bc58'),
  'dist': {'calculated': 88.52102010398153,
           'location': [144.9645504, -37.80818365]},
  'features': {'geometry': {'coordinates': [144.9645504, -37.80818365],
                            'type': 'Point'},
               'properties': {'N

### Check

In [820]:
a1 = [144.9654, -37.8076]
a2 = [144.9656988, -37.80619915]

distance_finder(a1, a2)

157.69509130741474 m


## Implement Automation

In [102]:
click = [144.96402401414252, -37.8133816331916]
click = [ round(elem, 14) for elem in click ]
click

[144.96402401414252, -37.8133816331916]

In [920]:
location = [144.965234, -37.8076]

In [183]:
pipeline = [
        {'$geoNear': {
            'near' : { 'type' : "Point", 'coordinates' : click },
             'distanceField': "dist.calculated",
             'maxDistance': 500,
             'includeLocs': "dist.location",
              'key': 'features.geometry.coordinates',
             'uniqueDocs': False,
             'spherical': True}},
    { "$unwind" : "$features" },
    { "$redact" : {   
        "$cond" : {
            "if" : { "$eq" : [ { "$cmp" : [ "$features.geometry.coordinates", "$dist.location" ] }, 0 ] },
            "then" : "$$KEEP",
            "else" : "$$PRUNE"
        }
    }
}
]

pprint.pprint(list(collection.aggregate(pipeline)))

[{'_id': ObjectId('5ccd8bcce39a5cbeb4b1c0f2'),
  'dist': {'calculated': 18.504773621888866,
           'location': [144.9638295, -37.81344503]},
  'features': {'geometry': {'coordinates': [144.9638295, -37.81344503],
                            'type': 'Point'},
               'properties': {'Number of seats': 54,
                              'Street address': '314-336 Bourke Street',
                              'Trading name': 'The Salon By Louis Roderer',
                              'Type': 'Cafe'},
               'type': 'Feature'},
  'type': 'FeatureCollection'},
 {'_id': ObjectId('5ccd8bcce39a5cbeb4b1c0f2'),
  'dist': {'calculated': 18.504773621888866,
           'location': [144.9638295, -37.81344503]},
  'features': {'geometry': {'coordinates': [144.9638295, -37.81344503],
                            'type': 'Point'},
               'properties': {'Number of seats': 99,
                              'Street address': '314-336 Bourke Street',
                              'T

In [276]:
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
pprint.pprint(list(collection.aggregate(pipeline)))
connection.close()

[{'_id': ObjectId('5ccd8bcce39a5cbeb4b1c0f2'),
  'dist': {'calculated': 18.504773621888866,
           'location': [144.9638295, -37.81344503]},
  'features': {'geometry': {'coordinates': [144.9638295, -37.81344503],
                            'type': 'Point'},
               'properties': {'Number of seats': 54,
                              'Street address': '314-336 Bourke Street',
                              'Trading name': 'The Salon By Louis Roderer',
                              'Type': 'Cafe'},
               'type': 'Feature'},
  'type': 'FeatureCollection'},
 {'_id': ObjectId('5ccd8bcce39a5cbeb4b1c0f2'),
  'dist': {'calculated': 18.504773621888866,
           'location': [144.9638295, -37.81344503]},
  'features': {'geometry': {'coordinates': [144.9638295, -37.81344503],
                            'type': 'Point'},
               'properties': {'Number of seats': 99,
                              'Street address': '314-336 Bourke Street',
                              'T

## Calling Cafe

In [278]:
click = [144.96518348629792, -37.81309237049503]

In [149]:
import pandas as pd

In [279]:
pipeline_cafe = [
        {'$geoNear': {
            'near' : { 'type' : "Point", 'coordinates' : click },
             'distanceField': "dist.calculated",
             'maxDistance': 3000,
             'includeLocs': "dist.location",
              'key': 'features.geometry.coordinates',
             'uniqueDocs': False,
             'spherical': True}},
    {"$match" : {"features.properties.Type":"Cafe"}},
    { "$unwind" : "$features" },
    { "$redact" : {   
        "$cond" : {
            "if" : { "$eq" : [ { "$cmp" : [ "$features.geometry.coordinates", "$dist.location" ] }, 0 ]},
            "then" : "$$KEEP",
            "else" : "$$PRUNE"
            }
        }
    }
]

pprint.pprint(list(collection.aggregate(pipeline_cafe)))

[{'_id': ObjectId('5ccd8bcce39a5cbeb4b1c0f2'),
  'dist': {'calculated': 16.197999502475984,
           'location': [144.9649993, -37.81309229]},
  'features': {'geometry': {'coordinates': [144.9649993, -37.81309229],
                            'type': 'Point'},
               'properties': {'Number of seats': 42,
                              'Street address': '191-205 Swanston Street',
                              'Trading name': 'Soul Cafe',
                              'Type': 'Cafe'},
               'type': 'Feature'},
  'type': 'FeatureCollection'},
 {'_id': ObjectId('5ccd8bcce39a5cbeb4b1c0f2'),
  'dist': {'calculated': 16.197999502475984,
           'location': [144.9649993, -37.81309229]},
  'features': {'geometry': {'coordinates': [144.9649993, -37.81309229],
                            'type': 'Point'},
               'properties': {'Number of seats': 30,
                              'Street address': '191-205 Swanston Street',
                              'Trading name':

In [274]:
a = collection.find(
   {
     "features.geometry.coordinates":
       { "$near" :
          {
            "$geometry": { 'type': "Point", "coordinates" : click },
            "$maxDistance": 1/111.12
          }
       },
        "features.properties.Type":"Cafe"
   }
)


for i in a:
    print(i)


In [7]:
#collection.delete_many({"type": "FeatureCollection"})

In [10]:
collection.find_one({"features.properties.sensor_id" : { "$exists" : True }}, {"_id":0})

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9651532, -37.8134944]},
   'properties': {'sensor_id': 1,
    'sensor_description': 'Bourke Street Mall (North)',
    'sensor_name': 'Bou292_T',
    'installation_date': '24/03/2009',
    'status': 'A',
    'note': '',
    'direction_1': 'West',
    'direction_2': 'East'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9651672, -37.81380667]},
   'properties': {'sensor_id': 2,
    'sensor_description': 'Bourke Street Mall (South)',
    'sensor_name': 'Bou283_T',
    'installation_date': '30/03/2009',
    'status': 'A',
    'note': '',
    'direction_1': 'East',
    'direction_2': 'West'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9642949, -37.81101523]},
   'properties': {'sensor_id': 3,
    'sensor_description': 'Melbourne Central',
    'sensor_name': 'Swa295_T',
    'installation_date': '25/03/2009',
    's

In [11]:
collection.find_one({"features.properties.Type" : "Cafe"}, {"_id":0})

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9656241, -37.80080042]},
   'properties': {'Street address': '117 Grattan Street',
    'Trading name': "Roy's Cafe",
    'Number of seats': 21,
    'Type': 'Cafe'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9560169, -37.8134766]},
   'properties': {'Street address': '550 Lonsdale Street',
    'Trading name': "Donoghue's ",
    'Number of seats': 50,
    'Type': 'Cafe'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9634979, -37.81592157]},
   'properties': {'Street address': '97-103 Elizabeth Street',
    'Trading name': 'Subway',
    'Number of seats': 16,
    'Type': 'Cafe'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9556244, -37.81551353]},
   'properties': {'Street address': '188-202 King Street',
    'Trading name': 'La Di Da',
    'Number of seats': 80,
    'Type': 'Ca